# 1. Import library

In [1]:
import xlwings as xw
import sources.parts_list_0802 as pl
from sqlite3 import connect
import pandas as pd
import os
from datetime import datetime as dt
import sources.logins as lg
import pathlib
import qrcode
from time import sleep
import xlwings as xw
from IPython.display import Image
import warnings
warnings.filterwarnings('ignore')

## 2. Select database

In [2]:
conn = connect('quotation.db')

In [ ]:
# check all table
pd.read_sql("SELECT name FROM sqlite_master WHERE type ='table'",conn)

In [ ]:
q='''
        SELECT max([UPDATE TIME]) as [update time],min([rma no.]) as rma_min,
        max([rma no.]) as rma_max FROM consolidated
    '''
pd.read_sql(q,conn)

### Copy file Smmary



In [ ]:
file_smm = lg.file_select(folder_name='inspection')


In [ ]:
# copy full_file to destination folder
import shutil
import pathlib
import os
def copy_selected_file(des_folder,file_name):

#     des_folder = 'files'
    ans = str(input(f'Default folder "{des_folder}"(N) or change to folder:'))
    path = pathlib.Path().absolute()
    if ans !='n': des_folder = ans
    des_path = os.path.join(path,des_folder)
    shutil.copy(file_name,os.path.join(des_path,file_name.split('\\')[-1]))
    print(f'Copy file name {file_name}  to  {des_path} completed')

In [ ]:
copy_selected_file(des_folder = '',file_name = file_smm)

# filter Serial No


In [ ]:
def sqlite_power(x,n):
    return int(x)**n
conn.create_function("power", 2, sqlite_power)

In [ ]:
q = '''
        SELECT POWER(4,2)
    '''
pd.read_sql(q,conn)

In [ ]:
# SELECT ALL RMA EMPTY REPORT_ID
q = '''
        SELECT rma, customer,model,sn,report_id,status,[Exfm Status]
        FROM m_list
        WHERE report_id IS NULL
        
    '''
pd.read_sql(q,conn)

In [ ]:
# copy from folder_name to inspection
lg.backups(folder_name,'inspection')

In [ ]:
# delete file
path = pathlib.Path().absolute()
# folder_name = 'inspection'
folder_name = 'reports_' + dt.now().strftime('%y%m%d')
for file in os.listdir(folder_name):
    os.remove(os.path.join(path,folder_name,file))
    print(f'Removed file {file} successful.')

## Create TR/WTY

In [ ]:
conn = connect('quotation.db')

In [3]:
# CREATE list_rma
# tao list rma
a = pl.parts_list()
sn_list='''
kc643k241
8g403k185
kc643k008



'''
list_rma = a.list_rma(conn,sn_list.strip(),'\n')
list_rma = list_rma.sort_values(by=['rma'])
display(list_rma)
for sn in sn_list.strip().split('\n'):
    if sn not in list(list_rma['SERIAL_NO']):
        print(sn)

,rma,CUSTOMER_NAME,SERIAL_NO,MODEL,Approval,REPAIR_STATUS,IN_INSPECT_USER_NAME
0,FMSV2023090054,Hanoi Oncology Hospital,KC643K241,EC-530WL3,None,Inspection,Nguyen Tuan Minh
1,FMSV2023090059,Hung Viet Cancer Hospital,8G403K185,EG-760Z,None,Inspection,Le Van Hoan
2,FMSV2023090062,Hoan My Da Nang Hospital,KC643K008,EC-530WL3,None,Inspection,hoang


kc643k241
8g403k185
kc643k008


In [4]:
# add report to summary file

r_type = (input('Type of report tr/[wty]: ') or 'wty').lower() # 'wty' 
d = pl.summary_report()
summary_file = d.read_summary_data('2023_FFVN Service Endo_Inspection, WTY & TR.xlsx')
gdkt = d.read_summary_data(file_name = '2023_FFVN Service Endo_Inspection, WTY & TR.xlsx',mode=r_type)

# tr_lr = gdkt.range('A' + str(gdkt.cells.last_cell.row)).end('up').row # tr

i_report = 0
q_gdkt = {}
gdkt_lr = summary_file[r_type][0]
report_num =summary_file[r_type][1]
report_id = {}
for rma in list_rma['rma']:
    
    i_report +=1
    report_num += 1
    info = []
    part_list= []
    c = pl.technical_report()
    info,part_list = c.report_info(rma,conn)
    model = info['MODEL'][0]
    sn = info['SERIAL_NO'][0]
   
    # record to file summary
    
    gdkt.range('A' + str(gdkt_lr + i_report)).value = report_num
    
    wn = str(info['web_name'][0]).replace('Bệnh viện','BV')
    wn = wn.replace('Phòng Khám','PK')
    
    
    gdkt.range('B' + str(gdkt_lr + i_report)).value = info['CUSTOMER_NAME'][0]
    
    gdkt.range('C' + str(gdkt_lr + i_report)).value = model.replace(' V2','')
    gdkt.range('D' + str(gdkt_lr + i_report)).value = sn
    
    
    
    #add zero to report number
    if len(str(report_num))==1: report_num_str = f'00{report_num}'
    if len(str(report_num))==2: report_num_str = f'0{report_num}'
    if len(str(report_num))==3: report_num_str = f'{report_num}'
    
    # not number
#     report_num_str = 'xxx'
    
#     gdkt.range('E' + str(gdkt_lr + i_report)).value = d.issues(conn,rma) #gdkt
    if r_type == 'tr':
        rpid = dt.now().strftime('%y%m') + report_num_str + 'TR' #tr 
        gdkt.range('J' + str(gdkt_lr + i_report)).value = rma
    
    elif r_type == 'wty':
        rpid = 'FFVN-' + dt.now().strftime('%y%m') + report_num_str
        gdkt.range('M' + str(gdkt_lr + i_report)).value = rma
    
    gdkt.range('E' + str(gdkt_lr + i_report)).value = rpid
    
    report_id.update({sn:rpid})
    sum_info = d.summary_info(conn,rma) # gdkt
#     gdkt.range('F' + str(gdkt_lr + i_report)).value = sum_info['first_name'][0] #gdkt 
    gdkt.range('F' + str(gdkt_lr + i_report)).value = d.issues(conn,rma) # tr
#     gdkt.range('G' + str(gdkt_lr + i_report)).value ="'" +  sum_info['date_installed'][0]
#     gdkt.range('H' + str(gdkt_lr + i_report)).value ="'" + sum_info['receive'][0]
#     gdkt.range('I' + str(gdkt_lr + i_report)).value ="'" +  sum_info['inspect_date'][0]
    q_gdkt.update({rma:(info,part_list)})
    
print('Done!!!')
report_num = int(list(report_id.values())[0][-2:])
report_num

Type of report tr/[wty]: 
Done!!!


46

In [5]:
report_id,report_num

({'KC643K241': 'FFVN-2310046',
  '8G403K185': 'FFVN-2310047',
  'KC643K008': 'FFVN-2310048'},
 46)

In [6]:
report_num = int(list(report_id.values())[0][-2:])
report_num

46

In [7]:
#run all  list rma

for rma in list_rma['rma']:
    issue = str(d.issues(conn,rma))
    c = pl.technical_report()
    info,part_list = c.report_info(rma,conn)
    part_list
    info
    q = f'''
            SELECT c.[rma no.] ,strftime('%d/%m/%Y',c.[date installed]) AS [date_installed],
            c.[Scope Connect Count] as [scope_count],c.[update user],strftime('%d/%m/%Y',
            c.[Last Repair(Shipping)]) as last_repair,e.location,e.leader,
            strftime('%d/%m/%Y',c.recieve_date) as receive,[defect note],[Last RMA No.],strftime('%d/%m/%Y',[create]) as create_date
            FROM consolidated c
            LEFT JOIN engineers e ON C.[IN_INSPECT_USER_NAME] = e.exfm_name
            WHERE C.[RMA NO.]='{rma}'
        '''
    add_info = pd.read_sql(q,conn)
    
    #add zero to report number
    if len(str(report_num))==1: report_num_str = f'00{report_num}'
    if len(str(report_num))==2: report_num_str = f'0{report_num}'
    if len(str(report_num))==3: report_num_str = f'{report_num}'
    
    #not number
#     report_num_str = 'xxx'
    
    wb = xw.Book('templates\\Report_Template.xlsx')
    tp = wb.sheets('Report')
    folder_name = 'reports_' + dt.now().strftime('%y%m%d')

    path = pathlib.Path().absolute()
    try:
        os.mkdir(folder_name)
        print(f'folder {folder_name} was created.')
    except:
        print(f'Folder {folder_name} exists')
    if r_type == 'tr':
        ref_no = f"{dt.now().strftime('FFVN-%y%m')}{report_num_str}TR"
    elif r_type =='wty':
        ref_no = f"{dt.now().strftime('FFVN-%y%m')}{report_num_str}"

    tp.range('C5').value = ref_no
    tp.range('C6').value = dt.now().strftime('%d-%b-%y')
    tp.range('E6').value = info['RMA No.'][0]
    tp.range('C7').value = info['CUSTOMER_NAME'][0]
    tp.range('C8').value = 'VIETNAM'
    if r_type == 'wty':
        tp.range('C13').value = 'YES'
    elif r_type == 'tr':
        tp.range('C13').value = 'NO'
    tp.range('C10').value = info['MODEL'][0]
    tp.range('C11').value = str(info['SERIAL_NO'][0]).upper()
    try:
        tp.range('C12').value = dt.strptime(add_info['date_installed'][0],'%d/%m/%Y')
    except Exception as e:
        print('None Installation Date ')
    tp.range('E13').value = add_info['Last RMA No.'][0]
    tp.range('B15').value = add_info['Defect Note'][0]
    tp.range('C25').value = dt.strptime(add_info['create_date'][0],'%d/%m/%Y')

    try:
        tp.range('B27').value = add_info['Defect Note'][0] + '\n' +'Used case: ' + str(int(add_info['scope_count'][0]))
    except:
        tp.range('B27').value = add_info['Defect Note'][0]

    tp.range('C28').value = info['IN_INSPECT_USER_NAME'][0]
    try:
        tp.range('C29').value = dt.strptime(info['inspect_date'][0],'%d/%m/%Y')
    except:
        print('Not inspection completed')
    plf = a.part_list_final(rma,conn)

    if tp.range('B37').value == 'Name / Date :': #check empty table
        #insert new rows
        for i in range(len(plf)-1):
            tp.range('33:33').insert()
            tp.range('32:32').copy()
            tp.range('33:33').paste('formats')
    #     total_price = sum(part_list_final['Dealer Price'])
        # fill in parts infor_mtion
        for i in range(len(plf)):

            tp.range('B' + str(i+32)).value = plf['part_num'][i]
            tp.range('C' + str(i+32)).value = plf['PART_DESCRIPTION'][i] 

            tp.range('D' + str(i+32)).value = plf['QUANTITY'][i]
            tp.range('E' + str(i+32)).value = plf['FFVN Price'][i]

        tp.range('E'+str(i+33)).value = f'=SUMPRODUCT(D32:D{str(i+32)},E32:E{str(i+32)})'
        tp.name = ref_no
        model_t = str(info['MODEL'][0]).replace('-','')
        model_t= model_t.replace('/','')
        sn_t = str(str(info['SERIAL_NO'][0]).upper()).replace('-','')
        sn_t = sn_t.replace('/','')
        issue = issue.replace(' ','')
        wb.save(f'{folder_name}\\{ref_no}-{model_t},{sn_t}-{issue}.xlsx')
        print(f'Export {wb.name} completed')
        print(f'{wb.name}')
        wb.close()
    else:
        print('table not empty')
    report_num = int(report_num) +1
print('Done')


folder reports_231002 was created.


,PART_DESCRIPTION,Vie,SAP,part_num,QUANTITY,FFVN Price,FFVN_AMOUNT,Dealer Price,Dealer_AMOUNT,rma,Priority
0,BENDING SECTION C643A ROHS,Bộ phận uốn cong,FW12G,342A1251557C,1,424.00,424.00,718.644068,718.644068,FMSV2023090054,2.1
1,WIRE ASSY,Dây cáp (thép),FW12G,56A1082711A,1,4.24,4.24,7.186441,7.186441,FMSV2023090054,5.1
2,PIPE SLEEVE,Ống nối (đồng),FW12G,42B593410A,4,2.12,8.48,3.593220,14.372881,FMSV2023090054,9.1
3,NUT ASS'Y,Đai ốc (đồng),FW12G,54A379237A,4,4.24,16.96,7.186441,28.745763,FMSV2023090054,9.2
4,RUBBER FOR BSA,Vỏ cao su,FW12G,68B1207810A,1,11.66,11.66,19.762712,19.762712,FMSV2023090054,9.8


0 part(s) no price
Export FFVN-2310046-EC530WL3,KC643K241-DWA,BSA.xlsx completed
FFVN-2310046-EC530WL3,KC643K241-DWA,BSA.xlsx
Folder reports_231002 exists


,PART_DESCRIPTION,Vie,SAP,part_num,QUANTITY,FFVN Price,FFVN_AMOUNT,Dealer Price,Dealer_AMOUNT,rma,Priority
0,CHA-G403A,Đầu thu hình,FW12G,10A1331996J,1,5160.08,5160.08,8745.898305,8745.898305,FMSV2023090059,1.0
1,BSA-G403A,Bộ phận uốn cong,F,830Y200631C,1,591.48,591.48,1002.508475,1002.508475,FMSV2023090059,2.1
2,PROTECTION TUBE A,Ống bảo vệ,FW12G,68A1333870A,1,279.84,279.84,474.305085,474.305085,FMSV2023090059,3.6
3,FCTINLET COVER RUBBER,Vòng cao su,F,350N102482C,1,11.66,11.66,19.762712,19.762712,FMSV2023090059,8.8
4,HANGING SLEEVE,Ống nối (đồng),F,347N130337,4,4.24,16.96,7.186441,28.745763,FMSV2023090059,9.1
5,RUBBER FOR BSA ROHS,Vỏ cao su,FW12G,68B11144712,1,49.82,49.82,84.440678,84.440678,FMSV2023090059,9.8


0 part(s) no price
Export FFVN-2310047-EG760Z,8G403K185-CHA,BSA.xlsx completed
FFVN-2310047-EG760Z,8G403K185-CHA,BSA.xlsx
Folder reports_231002 exists


,PART_DESCRIPTION,Vie,SAP,part_num,QUANTITY,FFVN Price,FFVN_AMOUNT,Dealer Price,Dealer_AMOUNT,rma,Priority
0,ISA-C643A,Bộ phận thân ống soi,FW12G,343A1251586D,1,3769.36,3769.36,6388.745763,6388.745763,FMSV2023090062,1.0
1,PIPE SLEEVE,Ống nối (đồng),FW12G,42B593410A,4,2.12,8.48,3.593220,14.372881,FMSV2023090062,9.1
2,NUT ASS'Y,Đai ốc (đồng),FW12G,54A379237A,4,4.24,16.96,7.186441,28.745763,FMSV2023090062,9.2


0 part(s) no price
Export FFVN-2310048-EC530WL3,KC643K008-ISA.xlsx completed
FFVN-2310048-EC530WL3,KC643K008-ISA.xlsx
Done


In [ ]:
#check report_id current in Master List
q =f'''
        SELECT rma,sn,report_id,status
        
        FROM m_list
        WHERE rma IN ({str(list(list_rma['rma']))[1:-1]})
    '''
pd.read_sql(q,conn)

In [ ]:
# UPDATE report id
cur = conn.cursor()
m_list = pd.read_sql('SELECT * FROM m_list',conn)
for sn in list_rma['SERIAL_NO']:
    rpid = report_id[sn]
    cur.execute(f"""
                    UPDATE m_list SET report_id = '{rpid}'
                    
                    WHERE sn = '{sn}'
    
                """)
conn.commit()
pd.read_sql(q,conn)

In [ ]:
lg.backups(folder_name,'inspection')

In [ ]:
#copy summary file to inspection folder
import shutil
path = pathlib.Path().absolute()
summary_file = '2023_FFVN Service Endo_Inspection, WTY & TR.xlsx'
org_file = os.path.join(path,summary_file)
des_file = os.path.join(path,'inspection',summary_file)
shutil.copy(org_file,des_file)

In [ ]:
# delete all file in report_yymmdd folder
file = lg.file_filter(path = folder_name)
print('-'*26)
for i in range(1,1+len(file)):
    os.remove(f'{folder_name}\\{file[i]}')
    print(f'Deleted {file[i]}')

#### Export Master List


In [ ]:
file_name = lg.file_select(folder_name = 'files', end_with = '.xlsm')
xw.Book(file_name)

In [ ]:
# final
q = '''SELECT * FROM m_list'''
m_list_final = pd.read_sql(q,conn)
m_list_final = m_list_final.drop(['TAT received','TAT PO','waiting for part'],axis=1)
dtmp = dt.now().strftime('%y%m%d')
file_name = f'ML_{dtmp}.xlsx'
m_list_final.to_excel(file_name,index = False)
xw.Book(file_name)